In [1]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/healthcare-dataset/healthcare_dataset.csv


## **Load the Data**

Load the data and print information about the data, summary statistics, and the first few rows.

In [2]:
healthcare_file_path = "/kaggle/input/healthcare-dataset/healthcare_dataset.csv"

# read in csv file with pandas
health_data = pd.read_csv(healthcare_file_path)

In [3]:
# print information about the dataframe
health_data.info()

# change column names to snake case
health_data.columns = (health_data.columns.str.replace(' ','_').str.lower())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55500 entries, 0 to 55499
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Name                55500 non-null  object 
 1   Age                 55500 non-null  int64  
 2   Gender              55500 non-null  object 
 3   Blood Type          55500 non-null  object 
 4   Medical Condition   55500 non-null  object 
 5   Date of Admission   55500 non-null  object 
 6   Doctor              55500 non-null  object 
 7   Hospital            55500 non-null  object 
 8   Insurance Provider  55500 non-null  object 
 9   Billing Amount      55500 non-null  float64
 10  Room Number         55500 non-null  int64  
 11  Admission Type      55500 non-null  object 
 12  Discharge Date      55500 non-null  object 
 13  Medication          55500 non-null  object 
 14  Test Results        55500 non-null  object 
dtypes: float64(1), int64(2), object(12)
memory usage: 6.4

In [4]:
# print the top few rows of the dataframe
health_data.head()

,name,age,gender,blood_type,medical_condition,date_of_admission,doctor,hospital,insurance_provider,billing_amount,room_number,admission_type,discharge_date,medication,test_results
0,Bobby JacksOn,30,Male,B-,Cancer,2024-01-31,Matthew Smith,Sons and Miller,Blue Cross,18856.281306,328,Urgent,2024-02-02,Paracetamol,Normal
1,LesLie TErRy,62,Male,A+,Obesity,2019-08-20,Samantha Davies,Kim Inc,Medicare,33643.327287,265,Emergency,2019-08-26,Ibuprofen,Inconclusive
2,DaNnY sMitH,76,Female,A-,Obesity,2022-09-22,Tiffany Mitchell,Cook PLC,Aetna,27955.096079,205,Emergency,2022-10-07,Aspirin,Normal
3,andrEw waTtS,28,Female,O+,Diabetes,2020-11-18,Kevin Wells,"Hernandez Rogers and Vang,",Medicare,37909.782410,450,Elective,2020-12-18,Ibuprofen,Abnormal
4,adrIENNE bEll,43,Female,AB+,Cancer,2022-09-19,Kathleen Hanna,White-White,Aetna,14238.317814,458,Urgent,2022-10-09,Penicillin,Abnormal


In [5]:
# check if any columns have missing values
for col in health_data.columns:
    if health_data[col].isnull().any():
        print(str(col) + " has missing values")
    else:
        print(str(col) + " is good")

name is good
age is good
gender is good
blood_type is good
medical_condition is good
date_of_admission is good
doctor is good
hospital is good
insurance_provider is good
billing_amount is good
room_number is good
admission_type is good
discharge_date is good
medication is good
test_results is good


## **Add a column for number of days admitted to the hospital**

 Use the date of admission and date of discharge to determine the total number of days the patient was in the hospital.

In [6]:
# convert 'date_of_admission' and 'discharge_date' columns to datetime
dt_cols = ['date_of_admission','discharge_date']

for col in dt_cols:
    health_data[col] = pd.to_datetime(health_data[col], errors='coerce')

# subtract 'date_of_admission' from 'discharge_date' to get total number of admission days
health_data['length_of_admission'] = (health_data['discharge_date'] - health_data['date_of_admission']).dt.days

In [7]:
# print summary statistics for numerical data
health_data.describe()

,age,date_of_admission,billing_amount,room_number,discharge_date,length_of_admission
count,55500.000000,55500,55500.000000,55500.000000,55500,55500.000000
mean,51.539459,2021-11-01 01:02:22.443243008,25539.316097,301.134829,2021-11-16 13:15:20.821621504,15.509009
min,13.000000,2019-05-08 00:00:00,-2008.492140,101.000000,2019-05-09 00:00:00,1.000000
25%,35.000000,2020-07-28 00:00:00,13241.224652,202.000000,2020-08-12 00:00:00,8.000000
50%,52.000000,2021-11-01 00:00:00,25538.069376,302.000000,2021-11-17 00:00:00,15.000000
75%,68.000000,2023-02-03 00:00:00,37820.508436,401.000000,2023-02-18 00:00:00,23.000000
max,89.000000,2024-05-07 00:00:00,52764.276736,500.000000,2024-06-06 00:00:00,30.000000
std,19.602454,NaN,14211.454431,115.243069,NaN,8.659600


In [8]:
# get list of categorical variables
s = (health_data.dtypes == 'object')
object_cols = list(s[s].index)
print('\nCategorical Columns:')
print(object_cols)

# check number of values for one-hot encoding categorical variables 
print('\nNumber of unique values for categorical data:')
print('Hospitals: ' + str(health_data['hospital'].nunique()))
print('Medical Conditions: ' + str(health_data['medical_condition'].nunique()))
print('Insurance Providers: ' + str(health_data['insurance_provider'].nunique()))
print('Medications: ' + str(health_data['medication'].nunique()))
print('Blood Types: ' + str(health_data['blood_type'].nunique()))
print('Admission Types: ' + str(health_data['admission_type'].nunique()))
print('Test Results: ' + str(health_data['test_results'].nunique()))
print('Gender: ' + str(health_data['gender'].nunique()))


Categorical Columns:
['name', 'gender', 'blood_type', 'medical_condition', 'doctor', 'hospital', 'insurance_provider', 'admission_type', 'medication', 'test_results']

Number of unique values for categorical data:
Hospitals: 39876
Medical Conditions: 6
Insurance Providers: 5
Medications: 5
Blood Types: 8
Admission Types: 3
Test Results: 3
Gender: 2


In [9]:
# set the target and predictor columns
y = health_data.length_of_admission
features = ['age','gender','blood_type','medical_condition','insurance_provider','admission_type','medication','test_results']
X = health_data[features]

## **Subset data for testing, training and validation**
One-hot encode categorical columns, then split data into the following:

30% testing

70% training (56% training, 14% validation)

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# one-hot encode categorical data
t = (X.dtypes == 'object')
X_object_cols = list(t[t].index)
OH_encoder = OneHotEncoder(handle_unknown='ignore',sparse_output=False)
OH_array = OH_encoder.fit_transform(X[X_object_cols])
OH_cols = pd.DataFrame(OH_array, columns=OH_encoder.get_feature_names_out(X_object_cols))
OH_cols.index = X.index

# remove categorical columns, replace with one-hot encoding
num_health_data = X.drop(X_object_cols, axis=1)
OH_X = pd.concat([num_health_data, OH_cols], axis=1)
OH_X.columns = OH_X.columns.astype(str)
OH_X.head()

,age,gender_Female,gender_Male,blood_type_A+,blood_type_A-,blood_type_AB+,blood_type_AB-,blood_type_B+,blood_type_B-,blood_type_O+,...,admission_type_Emergency,admission_type_Urgent,medication_Aspirin,medication_Ibuprofen,medication_Lipitor,medication_Paracetamol,medication_Penicillin,test_results_Abnormal,test_results_Inconclusive,test_results_Normal
0,30,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,62,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,76,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,28,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,43,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


In [11]:
# split into testing (30%) and training (70%)
X_train_full, X_test, y_train_full, y_test = train_test_split(OH_X, y, train_size = 0.7, test_size = 0.3, random_state=0)

# divide dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, train_size = 0.8, test_size = 0.2, random_state = 0)

## **Determine random forest model**
Check MAE for a few different random forest models to determine which is best

In [12]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# define 5 different models
model_1 = RandomForestRegressor(n_estimators=50, random_state=0)
model_2 = RandomForestRegressor(n_estimators=100, random_state=0)
model_3 = RandomForestRegressor(n_estimators=150, random_state=0)
model_4 = RandomForestRegressor(n_estimators=150, min_samples_split=10, random_state=0)
model_5 = RandomForestRegressor(n_estimators=150 ,min_samples_split=10, max_depth=5, random_state=0)

models = [model_1, model_2, model_3, model_4, model_5]

# function for determining MAE of each model to compare
def model_mae(model, X_t=X_train, X_v=X_val, y_t=y_train, y_v=y_val):
    model.fit(X_t, y_t)
    preds = model.predict(X_v)
    return mean_absolute_error(y_v, preds)

# print MAE for each model
for i in range(0,len(models)):
    mae = model_mae(models[i])
    print("Model " + str(i+1) + " MAE: " + str(mae))

Model 1 MAE: 7.500595857081572
Model 2 MAE: 7.460596827439685
Model 3 MAE: 7.447536634593777
Model 4 MAE: 7.472449112380399
Model 5 MAE: 7.517275456818036


Model 3 has the lowest MAE, so that one will be used in downstream analysis.

## **Permutation Feature Importance**

Check each feature to determine the contribution of the feature in the model's performance

In [13]:
import eli5
from eli5.sklearn import PermutationImportance
from copy import deepcopy
from sklearn.metrics import accuracy_score, r2_score

# map original categorical features to their one-hot encoded columns
grouped_features = {
    'Gender': [col for col in X_val.columns if col.startswith('gender_')],
    'Blood Type':  [col for col in X_val.columns if col.startswith('blood_')],
    'Admission Type': [col for col in X_val.columns if col.startswith('admission_type_')],
    'Medication': [col for col in X_val.columns if col.startswith('medication_')],
    'Test Results': [col for col in X_val.columns if col.startswith('test_results_')],
    'Medical Condition': [col for col in X_val.columns if col.startswith('medical_condition_')],
    'Insurance Provider': [col for col in X_val.columns if col.startswith('insurance_')]
}

numeric_features = ['age'] 

def permute_group(X, group_cols):
    X_new = X.copy()
    perm = np.random.permutation(X.index)
    X_new[group_cols] = X.loc[perm, group_cols].values
    return X_new

def permutation_importance(model, X_valid, y_valid,
                                  grouped_features, numeric_features,
                                  metric_fn, n_repeats=5):
    base_score = metric_fn(y_valid, model.predict(X_valid))
    importances = {}
    for group_name, columns in grouped_features.items():
        scores = []
        for _ in range(n_repeats):
            X_permuted = permute_group(X_valid, columns)
            score = metric_fn(y_valid, model.predict(X_permuted))
            scores.append(base_score - score)
        importances[group_name] = {
            'mean_importance': np.mean(scores),
            'std': np.std(scores)
        }

    # Individual numeric features
    for feature in numeric_features:
        scores = []
        for _ in range(n_repeats):
            X_permuted = permute_group(X_valid, [feature])
            score = metric_fn(y_valid, model.predict(X_permuted))
            scores.append(base_score - score)
        importances[feature] = {
            'mean_importance': np.mean(scores),
            'std': np.std(scores)
        }

    return pd.DataFrame(importances).T.sort_values('mean_importance', ascending=False)

permutation_importance(
    model_3, X_val, y_val,
    grouped_features,
    numeric_features,
    metric_fn=r2_score,
    n_repeats=10
)

2025-07-29 17:17:15.384531: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753809435.618023      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753809435.693023      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


,mean_importance,std
age,0.040178,0.004296
Blood Type,0.037265,0.004797
Insurance Provider,0.035206,0.003541
Medical Condition,0.032919,0.003746
Medication,0.028485,0.004181
Test Results,0.022872,0.002876
Admission Type,0.021134,0.002838
Gender,0.017388,0.004200
